In [9]:
import os
import uuid
import tempfile
import time
import re

from llm import get_llm
from prompt import story_request, generate_story, image_request, generate_image_prompt
from flux import generate_image

from docx import Document
from docx.shared import Inches
from dotenv import load_dotenv

In [10]:
load_dotenv()

True

In [16]:
# Function to generate the story
def inference(llm_instance, story_params):
    req = story_request(
        Age=story_params["Age"],
        Theme=story_params["Theme"],
        Pages=story_params["Pages"],
        Time=story_params["Time"],
        Tone=story_params["Tone"],
        Setting=story_params["Setting"],
        Moral=story_params["Moral"]
    )
    
    prompt_text = generate_story(req)
    print("\nGenerating story. Please wait...\n")

    response = llm_instance.invoke(prompt_text)
    
    return response.content

In [21]:
# ---------------------------
# Step 2. Parse the LLM output story into sections/pages.
# ---------------------------
def parse_story_sections(story_text):
    """
    Parses the LLM output story text into sections using markers enclosed in '**'.
    
    The story is expected to include markers like:
        **Title: The Amazing Adventure**
        **Page 1:**
        **Page 2:**
        **Ending:**
    
    Returns:
        sections (list of str): A list of section texts, each including its marker and content.
    """
    # The pattern matches markers between ** and **
    pattern = r'\*\*(.*?)\*\*\s*'
    
    matches = list(re.finditer(pattern, story_text, flags=re.DOTALL))
    sections = []
    
    for i, match in enumerate(matches):
        marker = match.group(1).strip()
        start = match.end()
        end = matches[i+1].start() if (i+1) < len(matches) else len(story_text)
        content = story_text[start:end].strip()
        section_text = f"{marker}\n\n{content}" if content else marker
        sections.append(section_text)
    
    return sections

In [22]:
# ---------------------------
# Step 3. Generate images for each page.
# ---------------------------
def generate_images_for_sections(sections, style="sketch"):
    """
    For each section in the story, generate an image using the provided image generation functions.
    
    Parameters:
        sections (list of str): The story sections.
        style (str): A style string for the image generation.
        
    Returns:
        image_paths (list of str): A list of file paths for the generated images.
    """
    image_paths = []
    
    for idx, section in enumerate(sections):
        print(f"Generating image for section {idx+1}...")
        img_req = image_request(style=style, bedtime_story_content=section)
        img_prompt = generate_image_prompt(img_req)
        image = generate_image(img_prompt)
        
        if image:
            temp_dir = tempfile.gettempdir()
            image_filename = os.path.join(temp_dir, f"section_{idx+1}_{uuid.uuid4().hex}.png")
            image.save(image_filename)
            image_paths.append(image_filename)
            print(f"Image for section {idx+1} saved as {image_filename}\n")
        else:
            print(f"Failed to generate image for section {idx+1}.\n")
            image_paths.append(None)
        
        time.sleep(1)  # Optional pause between image generations
    return image_paths


In [23]:
# ---------------------------
# Step 4. Save the formatted story (with images) to a Word document.
# ---------------------------
def save_story_to_docx(sections, image_paths, output_filename="bedtime_story.docx"):
    """
    Saves the story with formatted headings and images to a Word document.
    
    If the first section starts with "Title:", that section is used as the document title.
    Each subsequent section is processed so that:
      - The first line (marker) is added as a heading (if it matches known markers).
      - The remaining text is added as paragraph text.
      - The corresponding image is inserted below the text.
      
    Parameters:
        sections (list of str): The story sections.
        image_paths (list of str): File paths to images corresponding to each section.
        output_filename (str): The output Word file name.
    """
    document = Document()
    
    # If the first section is the title, use it as the document title.
    if sections and sections[0].startswith("Title:"):
        lines = sections[0].splitlines()
        title_line = lines[0].strip()  # e.g., "Title: The Amazing Adventure"
        title_text = title_line.replace("Title:", "").strip()
        document.core_properties.title = title_text
        document.add_heading(title_text, level=1)
        sections = sections[1:]
        if image_paths:
            image_paths = image_paths[1:]
    
    # Process remaining sections.
    for idx, section in enumerate(sections):
        lines = section.splitlines()
        if not lines:
            continue
        
        first_line = lines[0].strip()
        # If the first line matches common markers, add it as a heading.
        if any(first_line.startswith(marker) for marker in ["Opening Hook:", "Page", "Ending", "The End"]):
            document.add_heading(first_line, level=2)
            remaining_text = "\n".join(lines[1:]).strip()
            if remaining_text:
                document.add_paragraph(remaining_text)
        else:
            document.add_paragraph(section)
        
        # Insert the corresponding image (if available).
        if idx < len(image_paths) and image_paths[idx]:
            try:
                document.add_picture(image_paths[idx], width=Inches(4))
            except Exception as e:
                print(f"Error inserting image for section {idx+1}: {e}")
    
    document.save(output_filename)
    print(f"\n📖 Story saved to: {output_filename}")

In [24]:
# ---------------------------
# Main execution
# ---------------------------
if __name__ == '__main__':
    # Define user parameters for the story generation.
    user_inputs = {
        "Age": "10",
        "Theme": "adventure",
        "Pages": 10,
        "Time": 10,
        "Tone": "fun",
        "Setting": "forest",
        "Moral": "kindness"
    }
    
    # Check for the API key and initialize the LLM instance.
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        print("Error: OPENAI_API_KEY not found in environment variables.")
        exit(1)
    
    llm_instance = get_llm(OPENAI_API_KEY)
    
    # Generate the story from the LLM.
    story_text = inference(llm_instance, user_inputs)
    print("\nStory generated successfully!\n")
    
    # Parse the LLM output story into sections.
    sections = parse_story_sections(story_text)
    
    # Generate images for each section.
    image_paths = generate_images_for_sections(sections, style="sketch")
    
    # Save the formatted story and images into a Word document.
    save_story_to_docx(sections, image_paths)


Generating story. Please wait...


Story generated successfully!

Generating image for section 1...
Image generation status: Pending
Image for section 1 saved as /tmp/section_1_af270b9cd5084897ae8d95f96f75539d.png

Generating image for section 2...
Image generation status: Pending
Image for section 2 saved as /tmp/section_2_f7737df0b4014d2fb012cf35ee7d1ece.png

Generating image for section 3...
Image generation status: Pending
Image for section 3 saved as /tmp/section_3_30a5966bb28c41b5915f1dbe110b548f.png

Generating image for section 4...
Image generation status: Pending
Image for section 4 saved as /tmp/section_4_8e1a7537553e4fdbb0161e4429d5b210.png

Generating image for section 5...
Image generation status: Pending
Image for section 5 saved as /tmp/section_5_924f83922b244db58bc3587855a94a50.png

Generating image for section 6...
Image generation status: Pending
Image for section 6 saved as /tmp/section_6_07408d69e39b451f9c23605eedf40b60.png

Generating image for section 7...
Image